In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import scipy.stats
import sys
sys.path.insert(1, '/glade/u/home/lettier/analysis/')
import master_utils as myf
xr.set_options(keep_attrs=True)

In [ ]:
myvariables = ['TREFHT']
mytime = pd.date_range(start="1979-01-01",end="2018-12-31", freq='M')
eddir = '/glade/scratch/wriggles/archive/'
mydir = '/glade/work/lettier/NUDGE/'

In [ ]:
def wrangle_nudge (nudge_name, myvariables):
    ds_a = xr.open_mfdataset(eddir+nudge_name+'/atm/hist/*.h0.*')[myvariables]
    ds_b = xr.open_mfdataset(eddir+nudge_name+'_21C/atm/hist/*.h0.*')[myvariables]
    ds = xr.concat([ds_a,ds_b],dim='time')
    ds['time'] = mytime
    ds['names'] = nudge_name
    ds = ds.set_coords('names')
    
    return ds

In [ ]:
def wrangle_proc_nudge(nudge_name, myvariables, campdir):

    listds = []
    for var in myvariables:
        listds.append(xr.open_dataset('/glade/campaign/univ/'+campdir+'/sosi_nudge_era/'+nudge_name+'/atm/proc/tseries/month_1/'+nudge_name+'.cam.h0.'+var+'.197901-200512.nc').isel(lev=-1))
    ds_a = xr.merge(listds)

    listds = []
    for var in myvariables:
        listds.append(xr.open_dataset('/glade/campaign/univ/'+campdir+'/sosi_nudge_era/'+nudge_name+'_21C/atm/proc/tseries/month_1/'+nudge_name+'_21C.cam.h0.'+var+'.200601-201812.nc').isel(lev=-1))
    ds_b = xr.merge(listds)

    ds = xr.concat([ds_a,ds_b],dim='time')
    ds['time'] = mytime
    ds['names'] = nudge_name
    ds = ds.set_coords('names')
    ds = ds[myvariables]

    return ds

In [ ]:
def wrangle_lens (e, myvariables):
    ledir = '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/'

    ens = str(e)
    if e<10:
        ens = '0'+str(e)
    tmp = []
    print(ens)
    for var in myvariables:
        myfiles = sorted([ledir+var+'/'+f for f in os.listdir(ledir+var) if ('B20TRC5CNBDRD' in f or 'BRCP85C5CNBDRD' in f) and '0'+ens+'.cam.h0.'+var in f ])
        myfiles = [f for f in myfiles if '.192001-199912.nc' not in f and '208101-210012.nc' not in f]
        ds = xr.open_mfdataset(myfiles)[[var]].sel(time=slice('1979-02','2019-01'))
        ds['time'] = mytime
        tmp.append(ds)
    ds = xr.merge(tmp)
    ds['names'] = 'LENS'+ens
    ds = ds.set_coords('names')

    return ds

In [ ]:
def get_timeseries (ds):
    
    garea, _, _ = myf.grid_area_regll(ds.lat,ds.lon)
    tarea = xr.DataArray(garea, dims=['lat','lon'], coords = {'lat':ds.lat,'lon':ds.lon})
    mymasks = [tarea, tarea.where(tarea.lat>45.), tarea.where(tarea.lat>60.), tarea.where(tarea.lat>70.),
               tarea.where(tarea.lat<-45), tarea.where(tarea.lat<-60)]
    labels = ['Global','45-90N','60-90N','70-90N',
              '45-90S','60-90S']
    
    listds = []
    for m, mask in enumerate(mymasks):
        dsr = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
        dsr['region'] = labels[m]
        dsr = dsr.set_coords('region')
        listds.append(dsr)
    ds = xr.concat(listds,dim='region')
    name = str(ds.names.values)
  
    ds.attrs['desc'] = 'processed by Lettie Roach, Jan 2022'
    ds.to_netcdf(mydir+'processed/timeseries/'+str(name)+'.trefht_timeseries.1979-2018.nc')
    
    return ds    
    

Nudge runs

In [ ]:
for name in ['anom_nudge_era_60_arclo','anom_nudge_era_60_archi','anom_nudge_era_60']:
    ds = get_timeseries(wrangle_proc_nudge(name, myvariables, 'uwas0104'))

CESM1 ensemble

In [ ]:
for e in range(1,36,1):
    get_timeseries(wrangle_lens(e, myvariables))

Also process ERA Interim, HadCRUT and GISTEMP

In [ ]:
obds = xr.open_dataset('/glade/work/lettier/ERAI/mon/remap_cesmgrid/ei.moda.an.sfc.regn128sc.1979-2018_ptuv_remapcesmagrid.nc')
obds['time'] = mytime
obds = obds.rename({'t2m':'TREFHT'})[['TREFHT']]
obds['names'] = 'ERAI'
obds = obds.set_coords('names')
get_timeseries(obds)


In [ ]:
ds = xr.open_dataset('/glade/work/lettier/HadCRUT.5.0.1.0.analysis.anomalies.ensemble_mean.nc').sel(time=slice('1979','2018'))[['tas_mean']]
ds = ds.rename({'tas_mean':'TREFHT'})
ds['time'] = mytime
ds = ds.rename({'latitude':'lat','longitude':'lon'})
ds['names'] = 'HadCRUT5'
ds = ds.set_coords('names')
ds = get_timeseries(ds)

In [ ]:
ds = xr.open_dataset('/glade/work/lettier/gistemp1200_GHCNv4_ERSSTv5.nc').sel(time=slice('1979','2018'))[['tempanomaly']]
ds = ds.rename({'tempanomaly':'TREFHT'})
ds['time'] = mytime
ds['names'] = 'GISTEMPv4'
ds = ds.set_coords('names')
ds = get_timeseries(ds)